In [8]:
import os
import pandas as pd

# Lấy thư mục hiện tại đang chạy notebook
base_path = os.getcwd()
folder_name = "Tổng hợp ngân hàng"
folder_path = os.path.join(base_path, folder_name)

# Tên các file cần đọc
file_names = {
    "BCTT": "BCTT_TongHop.xlsx",
    "CDKT": "CDKT_TongHop.xlsx",
    "CSTC": "CSTC_TongHop.xlsx",
    "KQKD": "KQKD_TongHop.xlsx",
    "LC": "LC_TongHop.xlsx",
    "TM": "TM_TongHop.xlsx"
}

# Đọc dữ liệu
dfs = {}
for key, file in file_names.items():
    full_path = os.path.join(folder_path, file)
    if not os.path.exists(full_path):
        print(f"❌ KHÔNG tìm thấy file: {full_path}")
    else:
        dfs[key] = pd.read_excel(full_path)

# Kiểm tra đủ file chưa
if "KQKD" in dfs and "CDKT" in dfs:
    df_kqkd = dfs["KQKD"]
    df_cdkt = dfs["CDKT"]

    # Lọc và đổi tên chỉ tiêu
    df_income = df_kqkd[df_kqkd["Chỉ tiêu"] == "Thu nhập lãi thuần"].copy()
    df_income.rename(columns={"Giá trị": "Thu nhập lãi thuần"}, inplace=True)

    df_asset = df_cdkt[df_cdkt["Chỉ tiêu"] == "Tổng tài sản sinh lãi (bình quân)"].copy()
    df_asset.rename(columns={"Giá trị": "Tài sản sinh lãi"}, inplace=True)

    # Merge theo Bank và Quý
    df_merged = pd.merge(df_income, df_asset, on=["Bank", "Quý"], how="left")

    # Tính NIM
    df_merged["NIM (%)"] = (df_merged["Thu nhập lãi thuần"] * 4 / df_merged["Tài sản sinh lãi"]) * 100

    # Hiển thị kết quả
    df_nim = df_merged[["Bank", "Quý", "Thu nhập lãi thuần", "Tài sản sinh lãi", "NIM (%)"]].sort_values(by=["Bank", "Quý"])
    df_nim.reset_index(drop=True, inplace=True)
    display(df_nim.head(15))
else:
    print("❌ Thiếu dữ liệu KQKD hoặc CDKT để tính NIM.")


KeyError: 'Chỉ tiêu'